# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [15]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [16]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [17]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [18]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print( '* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.iter('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [19]:
document = ET.parse( './data/mondial_database.xml' )

# 10 countries with the lowest infant mortality rates

In [32]:
infantMortalityRates={}
for element in document.iterfind('country'):
    for subelement in element.iter('infant_mortality'):
        infantMortalityRates[element.find('name').text]= float(subelement.text)
        

In [33]:
infantMortalityRatesSorted= sorted(infantMortalityRates.items(), key=lambda x: x[1])

In [34]:
infantMortalityRatesSorted[0: 10] # 10 countries with the lowest infant mortality rates

[('Monaco', 1.81),
 ('Japan', 2.13),
 ('Norway', 2.48),
 ('Bermuda', 2.48),
 ('Singapore', 2.53),
 ('Sweden', 2.6),
 ('Czech Republic', 2.63),
 ('Hong Kong', 2.73),
 ('Macao', 3.13),
 ('Iceland', 3.15)]

# 10 cities with the largest population

In [35]:
citiesLargestPopulation={}
        
for element in document.iterfind('country'):
    for subelement in element.iter('city'):
        for subsubelement in subelement.iter('population'):
            #print(subsubelement.text)
            citiesLargestPopulation[subelement.find('name').text]=int(subsubelement.text)


In [36]:
citiesLargestPopulationSorted= sorted(citiesLargestPopulation.items(), key=lambda x: x[1], reverse=True) 

In [37]:
citiesLargestPopulationSorted[0:10]  #10 cities with the largest population

[('Shanghai', 22315474),
 ('Istanbul', 13710512),
 ('Mumbai', 12442373),
 ('Moskva', 11979529),
 ('Beijing', 11716620),
 ('São Paulo', 11152344),
 ('Tianjin', 11090314),
 ('Guangzhou', 11071424),
 ('Delhi', 11034555),
 ('Shenzhen', 10358381)]

# 10 ethnic groups with the largest overall populations (sum of best/ latest estimates over all countries)

In [38]:
ethnicGroupLargestPopulation={}
ethnicGroupLargestPopulationPercent={} 
ethnicGroupLargestPopulationNet={}
ethnic_group=[]
population=[]
for element in document.iterfind('country'):
    for subelement in element.iter('ethnicgroup'):
        a=element.find('ethnicgroup').attrib
        b=float(element.find('population[last()]').text)
        ethnicGroupLargestPopulationPercent[element.find('name').text]= a
        ethnicGroupLargestPopulation[element.find('name').text]= b
        d=element.find('ethnicgroup[1]').text
        c=b * (float(a['percentage'])/100)
        ethnic_group.append(d)
        population.append(c)
        ethnicGroupLargestPopulationNet[element.find('name').text]= c


In [44]:
import pandas as pd

ethnic_group= pd.Series(ethnic_group)
population= pd.Series(population)
ethnic_group
population

new= pd.concat([ethnic_group, population], axis=1 )
new1= new.drop_duplicates()
new2=new1.sort_values(by=1, ascending=False)
new3= new2.reset_index()
del new3['index']
new3.iloc[0:9]

,0,1
0,Han Chinese,1.245059e+09
1,Dravidian,3.027137e+08
2,European,2.549581e+08
3,African,1.626516e+08
4,Bengali,1.467769e+08
5,Japanese,1.265342e+08
6,Russian,1.146462e+08
7,Javanese,1.134560e+08
8,European,1.088867e+08


In [46]:
g= list(new3[0])
g=set(g)
h={}
result = new3.sort_values(by=0)

for name in g:
    sum=0
    for index, row in result.iterrows():
        if row[0] == name:
            sum=sum+ row[1]
        h[name]=sum

ELP= sorted(h.items(), key=lambda x: x[1], reverse=True)


In [47]:
ELP[0:10] # 10 ethnic groups with the largest overall populations 

[('Han Chinese', 1245058800.0),
 ('European', 441003291.0636),
 ('Dravidian', 302713744.25),
 ('African', 198605033.964),
 ('Bengali', 146776916.72),
 ('Mestizo', 141972918.46699998),
 ('Japanese', 126534212.00000001),
 ('Russian', 114646210.938),
 ('Javanese', 113456006.10000001),
 ('German', 79192720.332)]

# name and country of a) longest river, b) largest lake and c) airport at highest elevation

## a) longest river

In [50]:
import pandas as pd
name=[]
country=[]
length=[]
for element in document.iterfind('river'):
    name.append(element.find('name').text)
    temp= element.attrib
    country.append(temp['country'])
    for subelement in element.iter('length'):
        length.append(float(subelement.text))
#name
#country
#area
name= pd.Series(name)
country= pd.Series(country)
length= pd.Series(length)

river= pd.concat([name, country, length], axis=1 )
river.iloc[0:10]

,0,1,2
0,Thjorsa,IS,230.0
1,Joekulsa a Fjoellum,IS,206.0
2,Glomma,N,604.0
3,Lagen,N,322.0
4,Goetaaelv,S,93.0
5,Klaraelv,N S,460.0
6,Umeaelv,S,470.0
7,Dalaelv,S,520.0
8,Vaesterdalaelv,S,320.0
9,Oesterdalaelv,S,241.0


In [52]:
riverSorted= river.fillna(0)
riverSorted=riverSorted.sort_values(by=2, ascending=False)
riverSorted

riverSorted= riverSorted.reset_index()
del riverSorted['index']
riverSorted.iloc[0:10]

,0,1,2
0,Amazonas,CO BR PE,6448.0
1,Jangtse,CN,6380.0
2,Hwangho,CN,4845.0
3,Lena,R,4400.0
4,Akagera,EAT RWA EAU,4374.0
5,Mekong,CN LAO THA K VN,4350.0
6,Irtysch,R KAZ CN,4248.0
7,Niger,RMM RN WAN RG,4184.0
8,Missouri,USA,4130.0
9,Jenissej,R,4092.0


In [53]:
countryList=[]
countryCode=[]
for element in document.iterfind('country'):
    temp= element.attrib
    countryCode.append(temp['car_code'])
    countryList.append(element.find('name').text)
    
countryList
countryCode

countryList= pd.Series(countryList)
countryCode= pd.Series(countryCode)
CountryTally= pd.concat([countryCode, countryList], axis=1 )
CountryTally.iloc[0:10]

,0,1
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro
5,KOS,Kosovo
6,AND,Andorra
7,F,France
8,E,Spain
9,A,Austria


In [54]:
riverSorted1= riverSorted.merge(CountryTally, how='left', left_on=1, right_on=0)
riverSorted1['0_y']=riverSorted1[1]
riverSorted1

del riverSorted1[1]
del riverSorted1['1_x']
riverSorted1.columns= ['River', 'Length', 'Country Code', 'Country']
riverSorted1= riverSorted1.fillna('Multiple')
riverSorted1.iloc[0:10]

,River,Length,Country Code,Country
0,Amazonas,6448.0,CO BR PE,Multiple
1,Jangtse,6380.0,CN,China
2,Hwangho,4845.0,CN,China
3,Lena,4400.0,R,Russia
4,Akagera,4374.0,EAT RWA EAU,Multiple
5,Mekong,4350.0,CN LAO THA K VN,Multiple
6,Irtysch,4248.0,R KAZ CN,Multiple
7,Niger,4184.0,RMM RN WAN RG,Multiple
8,Missouri,4130.0,USA,United States
9,Jenissej,4092.0,R,Russia


In [55]:
riverSorted1.iloc[0] # longest river

River           Amazonas
Length              6448
Country Code    CO BR PE
Country         Multiple
Name: 0, dtype: object

## b) largest lake 

In [56]:
import pandas as pd
nameLake=[]
countryLake=[]
areaLake=[]
for element in document.iterfind('lake'):
    nameLake.append(element.find('name').text)
    temp= element.attrib
    countryLake.append(temp['country'])
    for subelement in element.iter('area'):
        areaLake.append(float(subelement.text))
#name
#country
#area
nameLake= pd.Series(nameLake)
countryLake= pd.Series(countryLake)
areaLake= pd.Series(areaLake)

lake= pd.concat([nameLake, countryLake, areaLake], axis=1 )
lake.iloc[0:10]

,0,1,2
0,Inari,SF,1040.0
1,Oulujaervi,SF,928.0
2,Kallavesi,SF,472.0
3,Saimaa,SF,4370.0
4,Paeijaenne,SF,1118.0
5,Mjoesa-See,N,368.0
6,Storuman,S,173.0
7,Siljan,S,290.0
8,Maelaren,S,1140.0
9,Vaenern,S,5648.0


In [58]:
lakeSorted= lake.fillna(0)
lakeSorted=lakeSorted.sort_values(by=2, ascending=False)
lakeSorted

lakeSorted= lakeSorted.reset_index()
del lakeSorted['index']
lakeSorted.iloc[0:10]

,0,1,2
0,Caspian Sea,R AZ KAZ IR TM,386400.0
1,Lake Manitou,CDN,82103.0
2,Rutanzige/Eduardsee,ZRE EAU,68870.0
3,Lake Ontario,CDN USA,59600.0
4,Lake Huron,CDN USA,57800.0
5,Dead Sea,IL JOR WEST,41650.0
6,Lake Victoria,EAT EAK EAU,32893.0
7,Etoscha Salt Pan,NAM,31792.0
8,Ozero Baikal,R,31492.0
9,Lake Eyasi,EAT,29600.0


In [59]:
lakeSorted1= lakeSorted.merge(CountryTally, how='left', left_on=1, right_on=0)
#lakeSorted1
lakeSorted1['0_y']= lakeSorted1[1]
del lakeSorted1[1]
del lakeSorted1['1_x']
lakeSorted1.columns= ['Lake', "Area", "Country Code", 'Country']
lakeSorted1= lakeSorted1.fillna('Multiple')
lakeSorted1.iloc[0:10]

,Lake,Area,Country Code,Country
0,Caspian Sea,386400.0,R AZ KAZ IR TM,Multiple
1,Lake Manitou,82103.0,CDN,Canada
2,Rutanzige/Eduardsee,68870.0,ZRE EAU,Multiple
3,Lake Ontario,59600.0,CDN USA,Multiple
4,Lake Huron,57800.0,CDN USA,Multiple
5,Dead Sea,41650.0,IL JOR WEST,Multiple
6,Lake Victoria,32893.0,EAT EAK EAU,Multiple
7,Etoscha Salt Pan,31792.0,NAM,Namibia
8,Ozero Baikal,31492.0,R,Russia
9,Lake Eyasi,29600.0,EAT,Tanzania


In [60]:
lakeSorted1.iloc[0] #  largest lake 

Lake               Caspian Sea
Area                    386400
Country Code    R AZ KAZ IR TM
Country               Multiple
Name: 0, dtype: object

## c) airport at highest elevation

In [61]:
import pandas as pd
nameAirport=[]
countryAirport=[]
elevationAirport=[]
for element in document.iterfind('airport'):
    nameAirport.append(element.find('name').text)
    temp= element.attrib
    countryAirport.append(temp['country'])
    for subelement in element.iter('elevation'):
        elevationAirport.append(subelement.text)
#name
#country
#area
nameAirport= pd.Series(nameAirport)
countryAirport= pd.Series(countryAirport)
elevationAirport= pd.Series(elevationAirport)
elevationAirport=elevationAirport.astype(float)
Airport= pd.concat([nameAirport, countryAirport, elevationAirport], axis=1 )
Airport.iloc[0:10]

,0,1,2
0,Herat,AFG,977.0
1,Kabul Intl,AFG,1792.0
2,Tirana Rinas,AL,38.0
3,Cheikh Larbi Tebessi,DZ,811.0
4,Batna Airport,DZ,822.0
5,Soummam,DZ,6.0
6,Tamanrasset,DZ,1377.0
7,Biskra,DZ,88.0
8,Mohamed Boudiaf Intl,DZ,691.0
9,Ain Arnat Airport,DZ,1024.0


In [62]:
AirportSorted= Airport.fillna(0)
AirportSorted= AirportSorted.sort_values(by=2, ascending=False)
AirportSorted

AirportSorted= AirportSorted.reset_index()
del AirportSorted['index']
AirportSorted.iloc[0:10]

,0,1,2
0,El Alto Intl,BOL,4063.0
1,Lhasa-Gonggar,CN,4005.0
2,Yushu Batang,CN,3963.0
3,Juliaca,PE,3827.0
4,Teniente Alejandro Velasco Astete Intl,PE,3311.0
5,Juana Azurduy De Padilla,BOL,2905.0
6,Mariscal Sucre Intl,EC,2813.0
7,Coronel Fap Alfredo Mendivil Duarte,PE,2719.0
8,Mayor General FAP Armando Revoredo Iglesias Ai...,PE,2677.0
9,Licenciado Adolfo Lopez Mateos Intl,MEX,2581.0


In [63]:
AirportSorted1= AirportSorted.merge(CountryTally, how='left', left_on=1, right_on=0)
del AirportSorted1[1]
del AirportSorted1['1_x']
AirportSorted1.columns= ['Airport Name', 'Elevation', 'Country Code', 'Country']
AirportSorted1.iloc[0:10]

,Airport Name,Elevation,Country Code,Country
0,El Alto Intl,4063.0,BOL,Bolivia
1,Lhasa-Gonggar,4005.0,CN,China
2,Yushu Batang,3963.0,CN,China
3,Juliaca,3827.0,PE,Peru
4,Teniente Alejandro Velasco Astete Intl,3311.0,PE,Peru
5,Juana Azurduy De Padilla,2905.0,BOL,Bolivia
6,Mariscal Sucre Intl,2813.0,EC,Ecuador
7,Coronel Fap Alfredo Mendivil Duarte,2719.0,PE,Peru
8,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0,PE,Peru
9,Licenciado Adolfo Lopez Mateos Intl,2581.0,MEX,Mexico


In [64]:
AirportSorted1.iloc[0] # airport at highest elevation

Airport Name    El Alto Intl
Elevation               4063
Country Code             BOL
Country              Bolivia
Name: 0, dtype: object